# **Objetivo del Cuaderno Jupyter**
- Análisis del comportamiento de los Accesos e Ingresos de las distintas Tecnologías.
- Basicamente acá compararemos información a Nivel Nacional, apoyandónos en las tablas `telefonia_fija`, `telefonia_movil`, `television` e `internet_pais` y trataremos de responder la pregunta `¿Porqué Internet?` y no otra tecnología 
<br>
<img src='assets/EDR.png'>
<br>

- Las tablas mostradas en la imagen `ya vienen sin valores Nulos`, y los diferentes `Tipos de Datos se observan en la imagen de arriba en las tablas del diagrama EDR`. 
- Para realizar parte del análisis cargue datos del archivo `tipos-de-cambio-historicos.csv` bajado de la pagina del `Banco Central de Argentina`.
- Al principio del Cuaderno hay una parte llamada `Preparando la Data` donde se hacen ajustes necesarios, para disponibilizar a la libreria de gráficos los datos de manera adecuada.  Sí desea ver sólo el análisis puede bajar hasta encontrar el TITULO `Gráficos`. 


Importando Librerias

In [1]:
import polars as pl
import duckdb

Importando funciones del archivo `utilidades.py`

In [2]:
from utils.utilidades import crear_tabla, crear_graf_pie, crear_grafica, crear_graf_df, crear_fecha_periodo

Cargo la Data de Ingresos y accesos de cada tecnología

In [3]:
internet_pais = pl.read_parquet('tablas/internet_pais.parquet')

telefonia_movil = pl.read_parquet('tablas/telefonia_movil.parquet')

telefonia_fija = pl.read_parquet('tablas/telefonia_fija.parquet')

television = pl.read_parquet('tablas/television.parquet')

***
# **Preparando los Datos**
Para poder observar algunas tendencias necesitamos hacer unos ajustes en los dataframes:
- La tabla `television` no tiene totalizados los accesos cada 100 habitantes u hogares, ni los ingresos
- Vamos a totalizar los accesos en `telefonia_movil`, para ver esa tendencia, ya que  la data de accesos por cada 100 habitantes que suministra este dataset es INCORRECTA.<br>
- Todos los datasets tienen data a partir de 2014 excepto `telefonia_movil` que tiene data de 2013. Igualaré para este análisis a todos los dataset (desde 2014), para ser justos en el análisis.

**A Considerar:**
- El tipo de Datos está reflejado en la Imagen que se observa arriba de las tablas
- El Tratamiento de Nulos (Donde los Hubo), se realizó en los archivos 01_extrayendo_internet.ipymb y 02_extrayendo_otros.ipynb



In [4]:
television = television.with_columns([
    (pl.col('accesos_sate_100_hab') + pl.col('accesos_susc_100_hab')).alias('accesos_100_hab'),
    (pl.col('accesos_sate_100_hog') + pl.col('accesos_susc_100_hog')).alias('accesos_100_hog'),
    (pl.col('ingresos_suscripcion_miles') + pl.col('ingresos_satelital_miles')).alias('ingresos_miles'),
])


mask = telefonia_movil['anio'] == 2013
telefonia_movil = telefonia_movil.filter(~mask)


telefonia_movil = telefonia_movil.with_columns([
    ((pl.col('total_accesos_post')) + (pl.col('total_accesos_pre'))).alias('total_accesos'),

    ((pl.col('llamadas_post_miles')) + (pl.col('llamadas_pre_miles'))).alias('total_llamadas'),

    ((pl.col('minutos_post_miles')) + (pl.col('minutos_pre_miles'))).alias('total_minutos'),

])

Los ingresos en estos Dataframe están en Miles de Pesos Argentinos. Voy a realizar una serie de acciones en las siguientes celdas para incorporar una columna que sean ingresos en Dolares. Para ello pienso ejecutar los siguientes pasos:
- Cargar el archivo `tipos-de-cambio-historicos.csv` bajado de la pagina del `Banco Central de Argentina`.
- Las tasas `dolar_estadounidense` y `dolar_referencia_com_3500` los voy a promediar.
- Dado que la información de ingresos que voy a manejar de las distintas tecnologías, están en periodos trimestrales, sacaré un Average de la tasa de cambio en cada Periodo. 
- Calculo en una nueva columna en cada Dataframe los ingresos en dolares 

In [5]:
tipos_cambios = pl.read_csv('datasets_iniciales/tipos-de-cambio-historicos.csv')
tipos_cambios['indice_tiempo','dolar_estadounidense','dolar_referencia_com_3500'].tail()

indice_tiempo,dolar_estadounidense,dolar_referencia_com_3500
str,str,str
"""2024-07-27""","""931.5""","""931.5833"""
"""2024-07-28""","""931.5""","""931.5833"""
"""2024-07-29""","""932.0""","""931.75"""
"""2024-07-30""","""932.5""","""932.25"""
"""2024-07-31""","""933.0""","""932.75"""


In [6]:
# Me voy a quedar solo con las columnas que necesito
tipos_cambios = tipos_cambios['indice_tiempo','dolar_estadounidense','dolar_referencia_com_3500']
# Creo una columna tipo Date y Casteo las columnas que tienen el precio del Dolar
tipos_cambios = tipos_cambios.with_columns([
    pl.col('indice_tiempo').cast(pl.Date).alias('fecha'),
    pl.col('dolar_estadounidense').cast(pl.Float32).alias('dolar_estadounidense'),
    pl.col('dolar_referencia_com_3500').cast(pl.Float32).alias('dolar_referencia_com_3500')
])

# De la columna tipo Date extraigo el Año y el Trimestre, y promedio entre las dos tasas de cambio
tipos_cambios = tipos_cambios.with_columns([
    pl.col("fecha").dt.quarter().alias("trimestre"),
    pl.col("fecha").dt.year().alias("anio"),
    ((pl.col('dolar_estadounidense') + pl.col('dolar_referencia_com_3500'))/2).alias('dolar_promedio')
])

# Creo una columna Periodo para poder tener identificado cada Trimestre
tipos_cambios = tipos_cambios.with_columns(
    (pl.col("anio").cast(pl.String) + "-0" + pl.col("trimestre").cast(pl.String)).alias("periodo")
)

# En dos Pasos me quedo con los periodos que corresponden con la data que manejo en las otras tablas
mask = tipos_cambios['anio'] < 2014
tipos_cambios = tipos_cambios.filter(~mask)

mask = (tipos_cambios['anio'] == 2024) & (tipos_cambios['trimestre'] > 1)
tipos_cambios = tipos_cambios.filter(~mask)

# Selecciono las columnas con que me quedo por ahora
tipos_cambios = tipos_cambios['fecha', 'anio', 'trimestre', 'periodo', 'dolar_promedio']
tipos_cambios.head()

fecha,anio,trimestre,periodo,dolar_promedio
date,i32,i8,str,f32
2014-01-01,2014,1,"""2014-01""",6.5195
2014-01-02,2014,1,"""2014-01""",6.544
2014-01-03,2014,1,"""2014-01""",6.5565
2014-01-04,2014,1,"""2014-01""",6.5565
2014-01-05,2014,1,"""2014-01""",6.5565


Solo Necesito el promedio de la divisa por Trimestre en los periodos de la data, me ayudo con sintáxis SQL

In [7]:
cambios_promedios = pl.DataFrame(duckdb.sql("""
SELECT periodo, AVG(dolar_promedio) as dolar_promedio 
FROM tipos_cambios
GROUP BY periodo
ORDER BY periodo;
""").to_arrow_table()
)
del tipos_cambios
cambios_promedios.head()

periodo,dolar_promedio
str,f64
"""2014-01""",7.607122
"""2014-02""",8.05488
"""2014-03""",8.296906
"""2014-04""",8.513784
"""2015-01""",8.688752


Creo la Columna Periodo en cada Dataframe y le agrego la columna `dolar_promedio` haciendo join en cada uno contra el dataframe `cambios_promedios` que acabamos de crear

In [8]:
# TABLA internet_pais
internet_pais = internet_pais.with_columns(
    (pl.col("anio").cast(pl.String) + "-0" + pl.col("trimestre").cast(pl.String)).alias("periodo")
)
df_fecha = crear_fecha_periodo(internet_pais)
internet_pais = internet_pais.join(cambios_promedios, on='periodo', how='left', coalesce=True)
internet_pais = internet_pais.join(df_fecha, on='periodo', how='left', coalesce=True)


# TABLA telefonia_movil
telefonia_movil = telefonia_movil.with_columns(
    (pl.col("anio").cast(pl.String) + "-0" + pl.col("trimestre").cast(pl.String)).alias("periodo")
)
df_fecha = crear_fecha_periodo(telefonia_movil)
telefonia_movil = telefonia_movil.join(cambios_promedios, on='periodo', how='left', coalesce=True)
telefonia_movil = telefonia_movil.join(df_fecha, on='periodo', how='left', coalesce=True)

# TABLA telefonia_fija
telefonia_fija = telefonia_fija.with_columns(
    (pl.col("anio").cast(pl.String) + "-0" + pl.col("trimestre").cast(pl.String)).alias("periodo")
)
df_fecha = crear_fecha_periodo(telefonia_fija)
telefonia_fija = telefonia_fija.join(cambios_promedios, on='periodo', how='left', coalesce=True)
telefonia_fija = telefonia_fija.join(df_fecha, on='periodo', how='left', coalesce=True)

# TABLA television
television = television.with_columns(
    (pl.col("anio").cast(pl.String) + "-0" + pl.col("trimestre").cast(pl.String)).alias("periodo")
)
df_fecha = crear_fecha_periodo(television)
television = television.join(cambios_promedios, on='periodo', how='left', coalesce=True)
television = television.join(df_fecha, on='periodo', how='left', coalesce=True)

Creo la columna `ingresos_dolar_miles` en cada dataframe

In [9]:
internet_pais = internet_pais.with_columns(
    (pl.col("ingresos_miles") / pl.col("dolar_promedio")).alias("ingresos_dolar_miles")
)

telefonia_movil = telefonia_movil.with_columns(
    (pl.col("ingresos_miles") / pl.col("dolar_promedio")).alias("ingresos_dolar_miles")
)

telefonia_fija = telefonia_fija.with_columns(
    (pl.col("ingresos_miles") / pl.col("dolar_promedio")).alias("ingresos_dolar_miles")
)

television = television.with_columns(
    (pl.col("ingresos_miles") / pl.col("dolar_promedio")).alias("ingresos_dolar_miles")
)

In [10]:
internet_pais.head(1)

id,anio,trimestre,ingresos_miles,accesos_100_hog,accesos_100_hab,mbps_media_bajada,periodo,dolar_promedio,fecha,ingresos_dolar_miles
i64,i64,i64,f64,f64,f64,f64,str,f64,date,f64
202401,2024,1,2.8042e8,78.89,24.79,139.15,"""2024-01""",834.279074,2024-03-30,336121.488304


In [11]:
telefonia_movil.head(1)

id,anio,trimestre,llamadas_post_miles,llamadas_pre_miles,sms_salientes,minutos_post_miles,minutos_pre_miles,ingresos_miles,accesos_100_hab,total_accesos_post,total_accesos_pre,total_accesos,total_llamadas,total_minutos,periodo,dolar_promedio,fecha,ingresos_dolar_miles
i64,i64,i64,f64,f64,i64,f64,f64,i64,f64,f64,f64,f64,f64,f64,str,f64,date,f64
201401,2014,1,4.001399e6,3.589181e6,18393198881,1.0128751e7,5.799223e6,15938353,148.07,8.224482e6,5.4725089e7,6.2949571e7,7.59058e6,1.5927974e7,"""2014-01""",7.607122,2014-03-30,2.0952e6


In [12]:
telefonia_fija.head(1)

id,anio,trimestre,accesos_hogares,accesos_comercial,accesos_gobierno,accesos_otros,accesos_100_hab,accesos_100_hog,ingresos_miles,periodo,dolar_promedio,fecha,ingresos_dolar_miles
i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,str,f64,date,f64
201401,2014,1,7897980,1065834,30917,567500,22.49197,74.044895,2.9161e6,"""2014-01""",7.607122,2014-03-30,383334.445151


In [13]:
television.head(1)

id,anio,trimestre,accesos_susc_100_hab,accesos_sate_100_hab,accesos_susc_100_hog,accesos_sate_100_hog,accesos_tot_suscripcion,accesos_tot_satelital,ingresos_suscripcion_miles,ingresos_satelital_miles,accesos_100_hab,accesos_100_hog,ingresos_miles,periodo,dolar_promedio,fecha,ingresos_dolar_miles
i64,i64,i64,f64,f64,f64,f64,i64,i64,f64,f64,f64,f64,f64,str,f64,date,f64
202401,2024,1,16.2,3.08,51.55,9.8,7601499,1445205,1.9079e8,9.4271e7,19.28,61.35,2.8506e8,"""2024-01""",834.279074,2024-03-30,341681.509553


Ahora Necesito Juntar una Data para unos Gráficos tipo Pie y una Tabla

In [14]:
dfs_4 = [internet_pais, telefonia_movil, telefonia_fija, television]
list_4 = ["Internet", "Telefonía Móvil", "Telefonía Fija", "Televisión"]

data_pies_tabla = []

for df, name in zip(dfs_4, list_4):
    mask = df['periodo'] == '2014-01'
    row = {}
    row['Tecnología'] = name
    row['Ingresos 2014'] = df.filter(mask)['ingresos_dolar_miles'][0]
    """ # A telefonia_movil LAS VARIACIONES de los accesos las mediré por `total_accesos` al resto las 
        # mediré por `accesos_100_hab` """
    row['Accesos 2014'] = df.filter(mask)['accesos_100_hab'][0] if name != 'Telefonía Móvil' else \
                          df.filter(mask)['total_accesos'][0]

    mask = (df['anio'] == 2024) & (df['trimestre'] == 1)
    mask = df['periodo'] == '2024-01'
    row['Ingresos 2024'] = df.filter(mask)['ingresos_dolar_miles'][0]
    row['Accesos 2024'] = df.filter(mask)['accesos_100_hab'][0] if name != 'Telefonía Móvil' else \
                          df.filter(mask)['total_accesos'][0]

    data_pies_tabla.append(row)


'''Nota: Aunque en Accesos en la Tecnología "Telefonía Movil", tiene unos numeros que son grandes con respecto
         a las demás tecnologías, con esta data, se utilizará para medir variaciones porcentuales. Es decir cuanto
         a variado los accesos en 2024 con respecto a los que habían en 2014. Lo que pasa es que desconfio de la data de
         "accesos_100_hab" para "telefonia_movil" '''
df_pies_tabla = pl.DataFrame(data_pies_tabla)
df_pies_tabla

Tecnología,Ingresos 2014,Accesos 2014,Ingresos 2024,Accesos 2024
str,f64,f64,f64,f64
"""Internet""",392271.130796,15.050104,336121.488304,24.79
"""Telefonía Móvil""",2.0952e6,6.2949571e7,922335.714509,6.2854079e7
"""Telefonía Fija""",383334.445151,22.49197,159568.635775,14.719928
"""Televisión""",736245.032393,18.305607,341681.509553,19.28


Variables comunes para los Gráficos

In [15]:
# Listas de Dataframes, Nombres de Graficos y Colores Para los Gráficos con 4 Lineas
dfs_4 = [internet_pais, telefonia_movil, telefonia_fija, television]
list_4 = ["Internet", "Telefonía Móvil", "Telefonía Fija", "Televisión"]
colors_4 = ['#22D3EE', '#38C172', '#F6993F', '#FFED4A']

# Listas de Datagrames, Nombres de Graficos y Colores Para los Gráficos con 3 Lineas
dfs_3 = [internet_pais, telefonia_fija, television]
list_3 = ["Internet", "Telefonía Fija", "Televisión"]
colors_3 = ['#22D3EE', '#F6993F', '#FFED4A']

***
# **Gráficos**
#### **A tomar en Cuenta:**
- Las variables `accesos_100_hab` y `accesos_100_hog` se refieren a la cantidad de accesos registrados (como suscripciones).  Por ejemplo en la primera `accesos_100_hab` es una manera de saber cuantas personas de cada 100 habitantes tienen registrado una linea de telefonia fija, o cuantas tienen un contrato de internet, etc.  Sí analizamos `es el porcentaje` de personas de la población que tienen contratado un servicio.  En la data de `telefonia_movil` en la hoja de `penetracion` estos números `son mayores a 100` por eso considere que esa data aunque exista la posibilidad de que sea cierta, y no tengo data que demuestre lo contrario, prefiero estudiar la tendencia de accesos de `telefonia_movil` con otras variables.


### **Gráfico 1:**
Tendencias de los Ingresos de cada Tecnología medido en Pesos Argentinos en la última Decada

In [25]:
ancho_grafico = 900
titulo = 'Tendencias de Ingresos en Miles de Pesos Argentinos'
columna = 'ingresos_miles'
fig = crear_grafica(dfs_4, columna, titulo, list_4, colors_4, ancho_grafico)

config = {'displayModeBar': False}
fig.show(config=config)

**Nota:** Si observamos la linea `Telefonía Móvil` tenía un valor de `38,73` Miles de Millones para el año `2018` y de `769,48` Miles de Millones para el año `2024`. En sólo 6 años ha tenido un crecimiento de `1986 %`, lo cual me  prende una alarma en el análisis. Por eso decidí añadir data del Banco Central de Argentina, para llevar estos números a una divisa de referencia internacional.

### **Gráfico 2**
Tendencias de los Ingresos de cada Tecnología medido en Dolares en la última Decada

In [19]:
ancho_grafico = 900
titulo = 'Tendencias de Ingresos en Miles de Dolares'
columna = 'ingresos_dolar_miles'
fig = crear_grafica(dfs_4, columna, titulo, list_4, colors_4, ancho_grafico)

config = {'displayModeBar': False}
fig.show(config=config)
# fig.show()

Acá se observa una realidad muy distinta. En Dolares todas las tecnologías han bajado su nivel de Ingresos considerablemente. Este bajón de Ingresos puede deberse a varias cuasas:
- Situación de `inflación/devaluación` local, en ciertos periodos de tiempo del análisis.
- Incorporación de Nuevos actores al mercado, lo que deriva en una mayor `oferta` y competencia, para conquistar una parte del mercado.
- Avances de la Ciencia en este periodo de tiempo, lo que puede derivar en que ahora la `masificación de la tecnología` sea mas barata para agentes prestadores de los distintos servicios, y a su vez estos últimos puden ofrecer planes (en dolares) mas económicos.
- En línea con el punto anterior, recordar que para los prestadores de servicio, el ofrecer estas tecnologías es tratado como `economía de escala`, ya que necesitan una fuerte Inversión Inicial que no varia, tengan o no tengan clientes (en el caso de Tecnologías que requieran de Tendidos de Cableado). <br>
<br>
**Nota:** Como próximos pasos antes de una inversión tocaría hacer lo siguiente:
- Investigar (quizas con economistas y/o expertos de Argentina), las perspectivas reales de la situación `devaluación/inflación` local del país.
- Investigar los costos de cada Tecnología, porque esta Data refleja los Ingresos, pero no se sabe cual es la situación de los costos, para poder manejar rentabilidad. `Es probable que así como han caído los ingresos, también hayan caído los costos, dado que hablamos del mercado tecnológico, pero en la data suministrada no hay data que pueda sustentar dicha hipótesis`.

### **Gráfico 3:**
Proporciones del mercado en Argentina, de estas tecnologías, en la última Decada, medido en Dolares

In [21]:
ancho_grafico = 900
fig = crear_graf_pie(df=df_pies_tabla, width=ancho_grafico)

config = {'displayModeBar': False}
fig.show(config=config)

### **Gráfico 4:**
Tendencias de los Accesos por cada 100 Hogares en Argentina en la última Decada

In [22]:
# Gráfica: accesos_100_hog
ancho_grafico = 900
titulo = 'Tendencias de Accesos por cada 100 Hogares'
columna = 'accesos_100_hog'
fig = crear_grafica(dfs_3, columna, titulo, list_3, colors_3, ancho_grafico)

config = {'displayModeBar': False}
fig.show(config=config)

### **Gráfico 5:**
Tendencias de los Accesos por cada 100 Habitantes en Argentina en la última Decada


In [23]:
# Gráfica: accesos_100_hab
ancho_grafico = 900
titulo = 'Tendencias de Accesos por cada 100 Habitantes'
columna = 'accesos_100_hab'
fig = crear_grafica(dfs_3, columna, titulo, list_3, colors_3, ancho_grafico)

config = {'displayModeBar': False}
fig.show(config=config)

### **Gráfico 6:**
Tendencia de la Cantidad de Accesos en la `Telefonía Movil`<br>
Nota: Al no poder utilizar la data suministrada de `accesos por cada 100 Habitantes` ya que es `data de la que no confio`, esta tendencia la estudiaremos con totales de accesos 

In [24]:
columns = ['total_accesos_post', 'total_accesos_pre', 'total_accesos']
names = ['Accesos Post-Pagos', 'Accesos Pre-Pagos', 'Total Accesos']
colors = ['#9fbc12', ' #045426 ', '#38C172']
titulo = 'Tendencia de Accesos de la Telefonía Movil'
ancho_grafico = 900
y_title = 'Accesos'

fig = crear_graf_df(columns=columns, names=names, colors=colors, width=ancho_grafico, 
                    y_title=y_title, df=telefonia_movil, titulo=titulo)

config = {'displayModeBar': False}
fig.show(config=config)

### **Gráfico 7:**
Tendencia de la Cantidad de llamadas en la `Telefonía Movil`

In [25]:
columns = ['llamadas_post_miles', 'llamadas_pre_miles', 'total_llamadas']
names = ['Llamadas Post-Pagos (miles)', 'Llamadas Pre-Pagos (miles)', 'Total Llamadas (miles)']
colors = ['#9fbc12', ' #045426 ', '#38C172']
titulo = 'Tendencia de Cantidad de llamadas en Telefonía Movil'
ancho_grafico = 900
y_title = 'Llamadas'

fig = crear_graf_df(columns=columns, names=names, colors=colors, width=ancho_grafico, 
                    y_title=y_title, df=telefonia_movil, titulo=titulo)

config = {'displayModeBar': False}
fig.show(config=config)

### **Gráfico 8:**
Tendencia de la Cantidad de minutos consumidos en la `Telefonía Movil`

In [26]:
columns = ['minutos_post_miles', 'minutos_pre_miles', 'total_minutos']
names = ['Minutos Post-Pagos (miles)', 'Minutos Pre-Pagos (miles)', 'Total Minutos (miles)']
colors = ['#9fbc12', ' #045426 ', '#38C172']
titulo = 'Tendencia de Cantidad de Minutos en Telefonía Movil'
ancho_grafico = 900
y_title = 'Minutos'

fig = crear_graf_df(columns=columns, names=names, colors=colors, width=ancho_grafico, 
                    y_title=y_title, df=telefonia_movil, titulo=titulo)

config = {'displayModeBar': False}
fig.show(config=config)

En la `telefonía Móvil` aunque la cantidad de accesos no ha variado prácticamente a lo largo de la última decada, la cantidad de llamadas y minutos consumidos han caído drásticamente. Esto no necesariamente es malo y puede deberse a que la masificación del smartphone, hace que el teléfono sea utilizado para otras cosas y no nada más para hablar. También en la actualidad se está hablando por otras vias distintas a las lineas normales (o convencionales), como el caso de `whatsapp` o cualquier otra plataforma que permita hablar utilizando sevicios de internet.  Para entender la evolución de estos cambios, pensemos sólo en el ejemplo de las `antiguas llamadas de larga distancia internacional`, hoy en dia deben ser un número despreciable en comparación con lo que eran hace apenas 2 decadas. 

In [27]:
"""VARIACION EN PENETRACION E INGRESOS POR TECNOLOGIA """
ancho = 500
alto = 130
fig = crear_tabla(df=df_pies_tabla, width=ancho, height=alto)

config = {'displayModeBar': False}
fig.show(config=config)

### **Conclusiones y Recomendaciones:**
- La variación de los ingresos en dolares en el mercado tecnológico Argentino, merece el estudio en dos vertientes:
    - Situación inflación/devaluación Local
    - Estudio real de la evolución de los costos de masificar las tecnologías (esto es a nivel internacional)
- De las tecnologías estudiadas el internet es por lejos la de mayor crecimiento, y la telefonía fija está condenada a desaparecer o disminuirse a niveles despreciables, en comparación con lo que fue en el pasado.
- El `internet` está siendo usado para `hablar` y para `ver televisión` y los `teléfonos` están siendo utilizados para `consumir internet`.
- La opción de la empresa de apostar por el internet, según estos datos parece la correcta, y sólo debería considerar, si acompañar o no, con la telefonía móvil. 

